# Question 3:

### *Is there a correlation between product category and customer region?*

## Define Hypotheses

For this, we first need to define our hypotheses:
<br>
<br>
    ***Null Hypothesis:*** There is no correlation between product category and customer region.
<br>
    ***Alternate Hypothesis:*** There is a statistically significant correlation between product category and customer region.

For reference, the relational structure of this dataset is provided below.
<img src='Northwind_ERD.png'>

In [ ]:
# For employees
c.execute("""SELECT Id, LastName, FirstName, Title, ReportsTo, 
          BirthDate, City, Region FROM 'Employee';""")
employee_df = pd.DataFrame(c.fetchall())
employee_df.columns = [x[0] for x in c.description]
display(employee_df)
print('employee_df\n',employee_df.info())

In [ ]:
# For customers
c.execute("""SELECT Id, CompanyName, ContactName, ContactTitle, 
          City, Country, Region FROM 'Customer';""")
customer_df = pd.DataFrame(c.fetchall())
customer_df.columns = [x[0] for x in c.description]
display(customer_df.head(10))
print('customer_df\n',customer_df.info())

In [ ]:
# For Orders

c.execute("""SELECT Id, CustomerId, EmployeeId, OrderDate, RequiredDate, 
          ShippedDate, ShipVia, Freight, ShipCity, ShipCountry, ShipRegion
          FROM 'Order';""")
order_df = pd.DataFrame(c.fetchall())
order_df.columns = [x[0] for x in c.description]
display(order_df.head(10))
print('order_df\n',order_df.info())

c.execute("SELECT * FROM 'OrderDetail';")
orderdetail_df = pd.DataFrame(c.fetchall())
orderdetail_df.columns = [x[0] for x in c.description]
display(orderdetail_df.head(10))
print('orderdetail_df\n',orderdetail_df.info())

In [ ]:
print("Customer Locations by Region\n----------------------------\n", customer_df.Region.value_counts())
print("\nOrder Destinations by Region\n----------------------------\n", order_df.ShipRegion.value_counts())